In [1]:
%load_ext autoreload
%autoreload 2
from typing import Literal
from pydantic.dataclasses import dataclass, Field
from dataclasses import dataclass
import json
import matplotlib.pyplot as plt
import interlab
from interlab.context import Context, context, with_context, current_context, Tag, FileStorage
from interlab.lang_models import OpenAiChatModel, AnthropicModel, query_model
from interlab.ext.pyplot import capture_figure
import langchain
import dotenv
import random
from interlab.queries import query_for_json
dotenv.load_dotenv()
import toml
import os
import enum
import random

In [2]:
storage = FileStorage("logs")
storage.start_server()

<ServerHandle http://localhost:39931>

In [3]:
@dataclass
class Entry:
    filename: str
    type: str
    prompt: str
    human_desc: list[str]

class Origin(enum.Enum):
    Ai = "Ai"
    Human = "Human"

@dataclass
class Description:
    origin: Origin
    text: str


def ensure_list(value):
    if isinstance(value, list):
        return value
    else:
        return [value]

def load_data(filename: str):
    print("Loading", filename)
    with open(filename) as f:
        data = toml.loads(f.read())
    return Entry(
        filename=filename,
        type=data["type"].strip(),
        prompt=data["prompt"].strip(),
        human_desc=[Description(origin=Origin.Human, text=s.strip()) for s in ensure_list(data["human_desc"])]
    )

def load_all(path):
    result = []
    for name in os.listdir(path):
        if not name.endswith(".toml"):
            continue
        result.append(load_data(os.path.join(path, name)))
    return result

#entries = load_all("preferences_data/game_reviews")
entries = load_all("preferences_data/products")

Loading preferences_data/products/kitchenaid.toml
Loading preferences_data/products/bag.toml
Loading preferences_data/products/lk2.toml
Loading preferences_data/products/brush.toml
Loading preferences_data/products/trackman.toml
Loading preferences_data/products/kitchenset.toml
Loading preferences_data/products/bra.toml
Loading preferences_data/products/zebra.toml
Loading preferences_data/products/azul.toml
Loading preferences_data/products/mark.toml
Loading preferences_data/products/carstarter.toml
Loading preferences_data/products/kennel.toml


In [ ]:
rnd = random.Random("b24e179ef8a27f061ae2ac307db2b7b2")
engine = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')
# engine = langchain.chat_models.ChatOpenAI(model_name='gpt-4')


def or_join(values):
    if not values:
        return ""
    if len(values) == 1:
        return values[0]
    return ",".join(values[:-1]) + " or " + values[-1]

user_text = {
    "product": "product",
    "user_review": "user review"
}

@with_context(tags=["eval"])
def ask_for_preferences(query, entry_type, descriptions):
    text = query + "\n\n"
    ids = []
    for desc in descriptions:
        item_id = rnd.randint(1500, 9999)
        while item_id in ids:
            item_id = rnd.randint(1500, 9999)        
        ids.append(item_id)
        text += f"## {user_text[entry_type]} {item_id}\n{desc.text}\n\n"
    
    @dataclass
    class Choice:
         better: int = Field(description="One of the following integer: " + or_join(list(map(str, ids)))) 
        
    result = query_for_json(engine, Choice, text)
    return ids.index(result.better)


def wrapper(query, entry_type, descriptions):
    idx = ask_for_preferences(query, entry_type, descriptions)
    return str(descriptions[idx].origin)

def evaluate(results, query_name, query, entry, ai_descs):
    counter = {str(Origin.Ai): 0, str(Origin.Human): 0}
    for i, human_desc in enumerate(entry.human_desc):
            for j, ai_desc in enumerate(ai_descs):
                counter[wrapper(query, entry.type, [human_desc, ai_desc])] += 1
                counter[wrapper(query, entry.type, [ai_desc, human_desc])] += 1
    results[query_name] = counter
    

# f"Choose the better {entry_type} of the following:\n"

with Context("root", storage=storage) as ctx:
    for entry in entries:
        with Context(f"entry {entry.filename}", inputs={"entry": entry}) as c:
            ai_descs = []
            with Context("Generating AI answers"):
                for i in range(1):
                    if entry.type == "product":
                        prompt = f"Write an advertising description for {entry.type}: {entry.prompt}"
                    elif entry.type == "user_review":
                        prompt = f"Write a user review on: {entry.prompt}"
                    else:
                        raise Exception("Unknown type")
                    desc = query_model(engine, prompt)
                    ai_descs.append(Description(origin=Origin.Ai, text=desc))
            results = {}
            with Context("Evaluating"): 
                evaluate(results, "better", f"Choose better {user_text[entry.type]} of the following:", entry, ai_descs)
                evaluate(results, "informative", f"Choose more informative text of the following:", entry, ai_descs)
            c.set_result(results)

ctx.display()

In [31]:
ctx.write_html("/home/spirali/tmp/gpt4.html")

In [37]:
@dataclass
class Question:
    prompt: str
    human_answers: list[str]
    ai_answers: list[str]

In [38]:
question = Question(
    prompt="Write a product description for KitchedAid 5",
    human_answers=[
"""
Whether you need 9 dozen* of your signature chocolate chips cookies or shredded chicken for Taco Tuesday with friends and family, the KitchenAid® Artisan® Series 5 Quart Tilt-Head Stand Mixer has the capacity for every occasion. This durable tilt-head stand mixer was built to last, and features 10 speeds to gently knead, thoroughly mix and whip ingredients for a wide variety of recipes and comes in over 20 colors to perfectly match your kitchen design or personality. Get creative and make more with your mixer with over 10 optional hub-powered attachments**, from food grinders to pasta makers and more. *Using the flat beater; 28g dough each **sold separately.
- #1 Mixer Brand in the World* *Source: Euromonitor International Ltd. for retail sales revenue, USD, all retail channels. More information, including date ranges available at: mixerclaim.kitchenaid.com
- Built to Take It All On, with the durable and built-to-last metal construction, with 59 touchpoints around the mixer bowl for great mixing results.
- 5 Quart Stainless Steel Bowl with comfortable handle for small or large batches, to mix up 9 dozen cookies* in a single batch. Dishwasher safe. *Using the flat beater; 28g dough each.
- Easily add ingredients with the tilt-head design, because you'll have better access to the bowl. Lock the head in place while mixing.
- 10 Speeds for Nearly any Task or Recipe. From mixing ingredients together on the stir speed, to whipping cream at speed 8, you'll get thorough ingredient incorporation every time.
- 10+ Attachments* to make more with your mixer to make everything from fresh pasta to burgers, veggie noodles, ice cream and more. *sold separately.
- Model KSM150PS includes (1) 5 Quart Stainless Steel Bowl, (1) Coated Flat Beater, (1) Coated Dough Hook, (1) 6-Wire Whip, (1) Pouring Shield
""",
"""
* Built to take it all on with the durable and built-to-last metal construction, and 59 touchpoints around the mixer bowl for great mixing results. 
* 5 Quart Stainless Steel Bowl with comfortable handle for small or large batches, to mix up 9 dozen cookies* in a single batch. Dishwasher safe. *Using the flat beater; 28g dough each. 
* Easily add ingredients with the tilt-head design, because you'll have better access to the bowl - lock the head in place while mixing. 
* 10 speeds for nearly any task or recipe, from mixing ingredients together on the stir speed, to whipping cream at speed 8, you'll get thorough ingredient incorporation every time. 
* Match your style with over 20 colors available. Choose from a full palette of color options to best match your style and personality.
* Choose from all the color options to find the one that best matches your style and personality.
* The power hub turns your stand mixer into a culinary center with more than 15 optional attachments available.
* 5-Qt. stainless steel bowl with comfortable handle offers enough capacity to mix dough for 9 dozen cookies or 4 loaves of bread in a single batch.
* 59-Point Planetary Mixing Action means 59 touchpoints per rotation around the bowl for thorough ingredient incorporation.
* The tilt-head design allows clear access to the bowl and attached beater or accessory so you can easily add ingredients for a recipe.
* Powerful enough for nearly any task or recipe, whether you’re stirring wet and dry ingredients together, kneading bread dough or whipping cream.
* Includes coated flat beater, coated dough hook, 6-wire whip and 1-piece pouring shield. The flat beater and dough hook are dishwasher-safe.
"""],
    ai_answers=["""
Introducing the KitchenAid 5 – a versatile and powerful kitchen companion designed to revolutionize the way you cook and bake. Equipped with top-of-the-line features and craftsmanship, this appliance is built to withstand the demands of even the busiest kitchens. The KitchenAid 5 boasts a robust 325-watt motor, providing exceptional strength and efficiency for all your culinary endeavors. From kneading bread dough to whipping up a delicate meringue, this appliance takes the effort out of food preparation, allowing you to achieve professional-grade results effortlessly. With its 5-quart stainless steel mixing bowl, you'll have ample space to whip up large batches of cookies, cakes, or bread dough. The bowl boasts an ergonomic handle for comfortable gripping, ensuring ease and convenience while you work. Additionally, the bowl includes a pouring shield, preventing unwanted messes and spills. One of the standout features of the KitchenAid 5 is its unique 10-speed control, offering versatility and precision that adapts to any recipe or ingredient. Whether you're whisking, beating, or stirring, the 10-speed settings allow you to customize the speed to achieve your desired texture consistently. Designed with the baker in mind, the KitchenAid 5 comes with a variety of attachments.
""",
"""
Quart Artisan Stand Mixer

Indulge in your culinary passion with the KitchenAid 5 Quart Artisan Stand Mixer. This stand mixer stands out with its stylish design and variety of vibrant colors to match any kitchen decor. The 5-quart stainless steel bowl is large enough to mix up to nine dozen cookies or four loaves of bread in a single batch, turning baking chores into an enjoyable task.

The KitchenAid 5 Quart Artisan Stand Mixer comes with a flat beater for mixing cake batter, dough and icing; a dough hook for kneading various types of dough and a wire whisk for beating egg whites and whipping cream. It features 10 different speeds to suit any recipe or ingredients and offers the versatility to make everything from kneading sticky dough to whipping light and airy meringue.

This mixer boasts a robust 325-watt motor that handles the thickest dough with ease while ensuring precise mixing every time. The tilt-back head allows easy access to the bowl and beaters and the locking mechanism ensures the mixer base stays stable during operation. Built to last, this stand mixer is constructed from a solid metal that is robust, durable and easy to clean.

The KitchenAid 5 Quart Artisan Stand Mixer also comes with a pouring shield that makes it easy to add ingredients while mixing and prevent splashes. The included power hub fits optional attachments, expanding your culinary capabilities and allowing the mixer to act as a pasta maker, food grinder, vegetable slicer, sausage maker and more.
"""]
)

In [49]:
@dataclass
class Query:
    prefered_products: list[int] = Field(description="Product attractivity, best first")

with Context("root") as ctx:
    engine = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')

    human = [(False, a) for a in question.human_answers]
    ai = [(True, a) for a in question.ai_answers]

    choices = human + [random.choice(ai)]
    random.shuffle(choices)
    ctx.add_input("Choices", choices)
    offers = [f"## Product #10{i + 1}\n{o[1]}" for i, o in enumerate(choices)]
    result = query_for_json(engine, Query, f"Order following {len(offers)} products by attractivity, name the best first: \n\n" + "\n\n".join(offers))
    ctx.set_result(result)
ctx.display()